In [2]:
# Import Dependencies\
import pandas as pd

In [6]:
zika_path = "ZikaData2015.xlsx"
df_ziki_raw = pd.read_excel(zika_data)
demograph_path = "PEP_2018_PEPTCOMP_with_ann.csv"
df_demo_raw_init = pd.read_csv(demograph_path)
df_demo_raw = df_demo_raw_init.iloc[1:]
df_demo_raw.head()

,GEO.id,GEO.id2,GEO.display-label,popchg4201072018,natinc4201072018,births4201072018,deaths4201072018,netmig4201072018,intlmig4201072018,dommig4201072018,popchg20172018,natinc20172018,births20172018,deaths20172018,netmig20172018,intlmig20172018,dommig20172018
1,0400000US01,01,Alabama,107733,65595,484329,418734,42748,37551,5197,12751,3791,57216,53425,9062,3344,5718
2,0400000US02,02,Alaska,27189,57767,92762,34995,-31287,17792,-49079,-2348,5955,10693,4738,-8351,2401,-10752
3,0400000US04,04,Arizona,779358,269300,703872,434572,508013,145998,362015,122770,24744,83550,58806,97575,14335,83240
4,0400000US05,05,Arkansas,97797,63009,314960,251951,34163,25253,8910,10828,6098,37191,31093,4735,2260,2475
5,0400000US06,06,California,2302522,1984436,4078895,2094459,333168,1043561,-710393,157696,196471,477145,280674,-38271,117797,-156068


,Class,Name,Test Score,Test Score Summary
0,Oct,Cyndy,90,A
1,Oct,Logan,56,F
2,Jan,Laci,72,C
3,Jan,Elmer,88,B
4,Oct,Crystle,98,A
5,Jan,Emmie,67,D
